PROBLEM 5

In [ ]:
import numpy as np

def E(u,v): #define error surface
  return (u * np.exp(v) - 2 * v * np.exp(-u)) ** 2

def grad_E(u,v): #calculate the gradient of E
  chain = u * np.exp(v) - 2 * v * np.exp(-u)
  dE_du = 2 * chain * (np.exp(v) + 2 * v * np.exp(-u)) #partial with respect to u
  dE_dv = 2 * chain * (u * np.exp(v) - 2 * np.exp(-u)) #partial with respect to v
  return dE_du, dE_dv

u, v = 1.0, 1.0 #set u and v
eta = 0.1 #set eta
target_error = 1e-14 #target error is 10^-14
iterations = 0

while (True):
  error = E(u,v)
  if (error < target_error): #break if error is less than the target error
    break
  dE_du, dE_dv = grad_E(u, v) #find the gradient
  u -= eta * dE_du #calculate new u and v values after gradient descent
  v -= eta * dE_dv
  iterations += 1

print(f'Iterations: {iterations}')

Iterations: 10


PROBLEM 6

In [ ]:
print(f'(u, v) = {u}, {v}')

(u, v) = 0.04473629039778207, 0.023958714099141746


PROBLEM 7

In [ ]:
num_iterations = 15 #15 full iterations or 30 steps
u, v = 1.0, 1.0
eta = 0.1

for _ in range(num_iterations):
  dE_du, dE_dv = grad_E(u, v)
  u -= eta * dE_du #step 1, adjusting u for gradient descent
  error = E(u,v)
  dE_du, dE_dv = grad_E(u, v)
  v -= eta * dE_dv #step 2, adjusting v for gradient descent
  error = E(u,v) #calculate new error

print(f'Error after {num_iterations} iterations: {error}')

Error after 15 iterations: 0.13981379199615315


PROBLEM 8

In [ ]:
from os import sched_getscheduler
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss

space = [-1, 1]
N = 100 #number of training points
iterations = 100
nu = 0.01 #learning rate
test_size = 10000
num_epochs = 100


def generate_line():
  p1 = np.random.uniform(*space, 2)
  p2 = np.random.uniform(*space, 2)
  slope = (p2[1] - p1[1]) / (p2[0] - p1[0]) if p2[0] != p1[0] else np.inf
  b = p1[1] - slope * p1[0]
  return lambda x: slope * x + b

def prepare_data(N, target_function):
  def target(point):
      return target_function(point[0])

  X = []
  Y = []
  for i in range(N):
    new_pt = np.random.uniform(*space, 2)  #generate random points
    y = target(new_pt)
    difference = y - new_pt[1]

    X.append(new_pt)
    Y.append(difference)

  targets = np.sign(Y)  #convert differences to -1 or 1
  return X, targets

def sgd(X, y, nu, num_epochs):
  model = SGDClassifier(loss = "log_loss", learning_rate="constant", eta0=nu, max_iter=1, warm_start=True)
  for epoch in range(num_epochs):
    model.partial_fit(X, y, classes=np.array([-1, 1]))
  return model

def calc_e_out(model, test_size, target_function):
  X_test, y_test = prepare_data(test_size, target_function)
  y_prob = model.predict_proba(X_test)
  return log_loss(y_test, y_prob)


e_out_total = 0
target_function = generate_line()
for run in range(iterations):
  X_train, y_train = prepare_data(N, target_function)
  model = sgd(X_train, y_train, nu, num_epochs)
  e_out = calc_e_out(model, test_size, target_function)
  e_out_total += e_out

avg_e_out = e_out_total / iterations

print(f'Average E_out: {avg_e_out}')

Average E_out: 0.16323845994612346


In [ ]:
import numpy as np

# Define the space for random sampling
space = [-1, 1]

# Function to define a random boundary line
def generate_line():
    p1 = np.random.uniform(*space, 2)
    p2 = np.random.uniform(*space, 2)
    slope = (p2[1] - p1[1]) / (p2[0] - p1[0]) if p2[0] != p1[0] else np.inf
    b = p1[1] - slope * p1[0]
    return lambda x: slope * x + b

# Prepare training data based on the generated line
def prepare_data(N, target_function):
    X = []
    Y = []
    for _ in range(N):
        new_pt = np.random.uniform(*space, 2)
        y = target_function(new_pt[0])
        difference = y - new_pt[1]
        X.append(new_pt)
        Y.append(difference)

    targets = np.sign(Y)
    return np.array(X), targets

# Logistic function
def logistic_func(z):
    """Logistic function."""
    return 1 / (1 + np.exp(-z))

#train logistic regression using stochastic gradient descent
def train_logistic_regression(X, y, learning_rate=0.01, tol=0.01, max_iter=1000):
    weights = np.zeros(X.shape[1])
    N = len(y)

    for epoch in range(max_iter):
        weights_prev = weights.copy()
        #random permutation of indices for SGD
        indices = np.random.permutation(N)
        for i in indices:
            z = np.dot(X[i], weights)
            prediction = logistic_func(z)
            error = y[i] - prediction
            weights += learning_rate * error * X[i]  #update weights

        #check for convergence
        if np.linalg.norm(weights - weights_prev) < tol:
            break

    return weights

#calculate cross-entropy loss
def cross_entropy_loss(X, y, weights):
    predictions = logistic_func(np.dot(X, weights))
    return -np.mean(y * np.log(predictions + 1e-15) + (1 - y) * np.log(1 - predictions + 1e-15))

# Evaluate model performance on a separate test dataset
def evaluate_model(weights, num_test_points=1000, target_function=None):
    X_test = np.random.uniform(*space, (num_test_points, 2))
    #generate test labels based on the target function
    Y_test = np.sign(target_function(X_test[:, 0]) - X_test[:, 1])
    predictions = np.where(logistic_func(np.dot(X_test, weights)) >= 0.5, 1, -1)
    return np.mean(predictions != Y_test)  #error rate


N = 100  #number of training points
num_runs = 100  #number of experiments
errors = []

target_function = generate_line()
for _ in range(num_runs):
    X, Y = prepare_data(N, target_function)
    weights = train_logistic_regression(X, Y)  #train the model
    E_out = evaluate_model(weights, target_function=target_function)  #evaluate the model
    errors.append(E_out)

average_error = np.mean(errors)
print(f"Average E_out over {num_runs} runs: {average_error:.6f}")


Average E_out over 100 runs: 0.152280


In [3]:
import numpy as np

# Define the space for random sampling
space = [-1, 1]

def generate_line():
    """Generate a line based on two random points."""
    p1 = np.random.uniform(*space, 2)
    p2 = np.random.uniform(*space, 2)
    slope = (p2[1] - p1[1]) / (p2[0] - p1[0]) if p2[0] != p1[0] else np.inf
    b = p1[1] - slope * p1[0]
    return lambda x: slope * x + b

def prepare_data(N, target_function):
    """Prepare random data points and their corresponding labels."""
    X = []
    Y = []
    for _ in range(N):
        new_pt = np.random.uniform(*space, 2)  # Generate random points
        y = target_function(new_pt[0])  # Get the y-value from the target function
        difference = y - new_pt[1]  # Compute difference from line
        X.append(new_pt)
        Y.append(difference)

    targets = np.sign(Y)  # Convert differences to -1 or 1
    return np.array(X), targets

def logistic_func(z):
    """Logistic function."""
    return 1 / (1 + np.exp(-z))

def train_logistic_regression(X, y, learning_rate=0.01, tol=0.01, max_iter=10000):
    """Train logistic regression using Stochastic Gradient Descent."""
    weights = np.zeros(X.shape[1])  # Initialize weights
    N = len(y)  # Number of samples
    epoch_count = 0  # To count the number of epochs

    for epoch in range(max_iter):
        epoch_count += 1
        # Random permutation of indices
        indices = np.random.permutation(N)
        for i in indices:
            z = np.dot(X[i], weights)  # Compute linear combination
            prediction = logistic_func(z)  # Apply logistic function
            error = y[i] - prediction  # Calculate error
            weights += learning_rate * error * X[i]  # Update weights

        # Check for convergence
        if np.linalg.norm(weights) < tol:
            break

    return epoch_count

def main(num_runs=10, N=100):
    """Run multiple experiments to measure average epochs for convergence."""
    epoch_counts = []

    for _ in range(num_runs):
        target_function = generate_line()  # Generate a new target line
        X, Y = prepare_data(N, target_function)  # Prepare data
        epochs = train_logistic_regression(X, Y)  # Train the model
        epoch_counts.append(epochs)  # Store the epoch count for this run

    average_epochs = np.mean(epoch_counts)  # Calculate average epochs
    print(f"Average number of epochs for convergence: {average_epochs:.2f}")

# Run the main function
main()


<ipython-input-3-f423f5b032e7>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Average number of epochs for convergence: 10000.00
